In [1]:
# PyData
import pandas
import numpy
from pandas import DataFrame, Series
from sklearn import datasets, linear_model, preprocessing, cross_validation
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge

# System
import datetime
import os
import math

# Graphing
#%matplotlib inline # Only works on Python 3 in the docker container
#import seaborn # Only works on Python 3 in the docker container

#os.environ['PYSPARK_PYTHON'] = 'python2'

%matplotlib inline

# Spark
import pyspark
from pyspark.sql import SQLContext

from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

sc = pyspark.SparkContext('local[8]')
sqlContext = SQLContext(sc)

In [2]:
df_basic = DataFrame.from_csv("train_features_basic.csv", index_col=['Date', 'Store'])
df_means = DataFrame.from_csv("train-features-predicted_mean.csv", index_col=['Date', 'Store'])

In [3]:
df_sales = DataFrame.from_csv("train.csv", index_col=['Date', 'Store'])[['Sales']]

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
df_features = df_means.join(df_basic)
train_df = df_sales.join(df_features)
train_df.head()

Sales  Sales_predicted  DayOfWeek  Open  Promo  \
Date       Store                                                   
2015-07-31 1       5263      5145.283582          5     1      1   
           2       6064      5115.882353          5     1      1   
           3       8314      8138.089552          5     1      1   
           4      13995     10275.776119          5     1      1   
           5       4822      5308.835821          5     1      1   

                  StateHoliday  SchoolHoliday  cal:dayOfMonth  cal:dayOfWeek  \
Date       Store                                                               
2015-07-31 1                 0              1              31              4   
           2                 0              1              31              4   
           3                 0              1              31              4   
           4                 0              1              31              4   
           5                 0              1              31              4   

                  cal:dayofyear  cal:month  cal:quarter  cal:weekofyear  \
Date       Store                                                          
2015-07-31 1                212          7            3              31   
           2                212          7            3              31   
           3                212          7            3              31   
           4                212          7            3              31   
           5                212          7            3              31   

                  cal:year  
Date       Store            
2015-07-31 1          2015  
           2          2015  
           3          2015  
           4          2015  
           5          2015

In [5]:
del df_features
del df_basic
del df_means
del df_sales

In [6]:
train_df['Sales_predicted'] = train_df['Sales_predicted'].fillna(train_df['Sales_predicted'].mean())

In [7]:
train_df['Sales_predicted'] = train_df['Sales_predicted'].map(lambda x: int(x))

In [8]:
train_df.reset_index(inplace=True)

In [9]:
del train_df['Date']

In [10]:
# Sales Predicted Difference %

In [11]:
train_df['Sales_mean_prediction_error'] = (train_df['Sales_predicted'] - train_df['Sales']) / train_df['Sales']

In [12]:
feature_columns = list(train_df.columns)
feature_columns.remove("Sales")
feature_columns = ['Sales'] + feature_columns
train_df = train_df[feature_columns]

In [13]:

train_df.head()

,Sales,Store,Sales_predicted,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year,Sales_mean_prediction_error
0,5263,1,5145,5,1,1,0,1,31,4,212,7,3,31,2015,-0.022421
1,6064,2,5115,5,1,1,0,1,31,4,212,7,3,31,2015,-0.156497
2,8314,3,8138,5,1,1,0,1,31,4,212,7,3,31,2015,-0.021169
3,13995,4,10275,5,1,1,0,1,31,4,212,7,3,31,2015,-0.265809
4,4822,5,5308,5,1,1,0,1,31,4,212,7,3,31,2015,0.100788


# Switch to Spark

In [14]:
df = sqlContext.createDataFrame(train_df)
# This relies on correct order: [0] is sales, [1:] are all features
df = df.map(lambda row: LabeledPoint(row[0], row[1:3])).toDF()


In [ ]:
# The slowest fucking thing ever
# featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=10).fit(df)


In [15]:
(trainingData, testData) = df.randomSplit([0.7, 0.3])


# Train a RandomForest model.
rf = RandomForestRegressor(numTrees=2, maxDepth=2, maxBins=2, featuresCol="features")

# Chain indexer and forest in a Pipeline
# pipeline = Pipeline(stages=[featureIndexer, rf])
pipeline = Pipeline(stages=[rf])


In [16]:

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)



Py4JJavaError: An error occurred while calling o92.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 6.0 failed 1 times, most recent failure: Lost task 7.0 in stage 6.0 (TID 13, localhost): java.lang.OutOfMemoryError: Java heap space
	at java.lang.reflect.Array.newArray(Native Method)
	at java.lang.reflect.Array.newInstance(Array.java:70)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:1670)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1344)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:1706)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1344)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:1997)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:1921)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:1798)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1350)
	at java.io.ObjectInputStream.defaultRe

In [ ]:
1+1

In [ ]:

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
# print "Root Mean Squared Error (RMSE) on test data = %g" % rmse

# Compute RMPSE
squares = predictions.rdd.filter(lambda x: x.label != 0).map(lambda x: ((x.label - x.prediction) / x.label) *  ((x.label - x.prediction) / x.label))

mean = squares.mean()
import math
math.sqrt(mean)

In [ ]:

math.sqrt(mean)

In [ ]:
# Eval

In [ ]:
df_basic = DataFrame.from_csv("test_features_basic.csv", index_col=['Date', 'Store'])
df_means = DataFrame.from_csv("test-features-predicted_mean.csv", index_col=['Date', 'Store'])

In [ ]:
del df_means['Unnamed: 0']

In [ ]:
df_test_features = df_means.join(df_basic)

In [ ]:
#del df_test_features['Id']
# XXX WE NEED THIS LATER BUT NEED TO REMOVE BEFORE GIVING TO SPARK?

In [ ]:
df_test_features.reset_index(inplace=True)
del df_test_features['Date']

In [ ]:
df_test_features.head()

In [ ]:
len(df_test_features.columns)

In [ ]:
df_test = sqlContext.createDataFrame(df_test_features)
# This relies on correct order: [0] is sales, [1:] are all features
df_test = df_test.map(lambda row: LabeledPoint(0, features=row[0:])).toDF()

In [ ]:
df_test.take(5)

In [ ]:
prediction = df_test_features[['Id']]

In [ ]:
prediction['Sales'] = model.transform(df_test).map(lambda x: x.prediction).collect()

In [ ]:
prediction.head()

In [ ]:

prediction.to_csv( "spark-v1.csv", index = False )

In [ ]:
model.transform(trainingData).map(lambda x: x.prediction).take(5)

In [ ]:
 trainingData.take(5)